# How NOT to write pandas code

To Step Up Your Pandas Game, read: 
- [5 lesser-known pandas tricks](https://towardsdatascience.com/5-lesser-known-pandas-tricks-e8ab1dd21431)
- [Exploratory Data Analysis with pandas](https://towardsdatascience.com/exploratory-data-analysis-with-pandas-508a5e8a5964)

In [1]:
from platform import python_version

import numpy as np
import pandas as pd

In [2]:
np.random.seed(42) # set the seed to make examples repeatable

In [3]:
print("python version==%s" % python_version())
print("pandas==%s" % pd.__version__)
print("numpy==%s" % np.__version__)

python version==3.7.3
pandas==0.25.3
numpy==1.17.4


In [4]:
size = 10000

cities = ["paris", "barcelona", "berlin", "new york"]

df = pd.DataFrame(
    {"city": np.random.choice(cities, size=size), "booked_perc": np.random.rand(size)}
)
df["id"] = df.index.map(str) + "-" + df.city
df = df[["id", "city", "booked_perc"]]
df.head()

,id,city,booked_perc
0,0-berlin,berlin,0.393636
1,1-new york,new york,0.473436
2,2-paris,paris,0.854547
3,3-berlin,berlin,0.340004
4,4-berlin,berlin,0.869650


In [5]:
%%timeit

suma = 0
for _, row in df.iterrows():
    suma += row.booked_perc

766 ms ± 20.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit

sum(booked_perc for booked_perc in df.booked_perc)

989 µs ± 18.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
%%timeit

df.booked_perc.sum()

92 µs ± 2.21 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
%%timeit

suma = 0
for _, row in df.iterrows():
    if row.booked_perc <= 0.5:
        suma += row.booked_perc

831 ms ± 25.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
df[df.booked_perc <= 0.5].booked_perc.sum()

724 µs ± 18.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%%timeit

df[(df.booked_perc <= 0.5) & (df.city == 'new york')].booked_perc.sum()

1.55 ms ± 10.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
%%timeit

for i in range(1, len(df)):
    df.loc[i, "perc_change"] =  (df.loc[i].booked_perc - df.loc[i - 1].booked_perc) / df.loc[i - 1].booked_perc

7.02 s ± 24.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit

df["perc_change"] = df.booked_perc.pct_change()

586 µs ± 17.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
%%timeit

for i, row in df.iterrows():
    if row.city == 'new york':
        df.loc[i, 'sales_factor'] = row.booked_perc * 2
    else:
        df.loc[i, 'sales_factor'] = 0

3.58 s ± 48.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit

def calculate_sales_factor(row):
    if row.city == 'new york':
        return row.booked_perc * 2
    return 0

df['sales_factor'] = df.apply(calculate_sales_factor, axis=1)

165 ms ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%%timeit 

df.loc[df.city == 'new york', 'sales_factor'] = df[df.city == 'new york'].booked_perc * 2
df.sales_factor.fillna(0, inplace=True)

3.03 ms ± 85.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%%timeit 

avg_by_city = {}
count_by_city = {}
first_booking_by_city = {}

for i, row in df.iterrows():
    city = row.city
    if city in avg_by_city:
        avg_by_city[city] += row.sales_factor
        count_by_city[city] += 1
    else:
        avg_by_city[city] = row.sales_factor
        count_by_city[city] = 1
        first_booking_by_city[city] = row['id']

for city, _ in avg_by_city.items():
    avg_by_city[city] /= count_by_city[city]

878 ms ± 21.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit

df.groupby('city').sales_factor.mean()
df.groupby('city').sales_factor.count()
df.groupby('city').id.first()

3.05 ms ± 65.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%%timeit

df.groupby("city").agg({"sales_factor": ["mean", "count"], "id": "first"})

4.5 ms ± 131 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Surprisingly, the third example is not the fastest, but it is more concise than the second example.
I would suggest you use the second approach when you need to speed up your code.